In [1]:
import sqlite3
import requests

In [2]:
EXAMPLES = """
        Example 1: "Barnadråp i Gävle" -> Infanticide.
        Example 2: "29-åring knivhuggen och senare avliden i Sundsvall ca 2 mån sen" -> Stabbing.
        Example 3: "Nyheter i Stan" -> No crime
        Example 4: "Poliser ska ha ny uniform snart" -> No crime
        Example 5: "Misshandel i Nordstan utreds av polisen" -> Assault. 
        Example 6: "Polisen bombhotades" -> bomb threat
        Example 7: "Katter halshuggna, gärningsmän ska straffas" -> Animal abuse
        Example 8: "'Skottlossning i Malmö i natt'."
    """


def create_prompts(threads: list) -> list[tuple[str]]:
    return [(
        thread[0],
        f"{EXAMPLES}\nDoes the following Swedish sentence contain a crime? Reply strictly with the identified crime or 'No crime' and nothing else:\n'{thread[0]}'"
        )
        for thread in threads
        ]

def gpt4all_request(prompt: str) -> str:
   r = requests.post("http://localhost:4891/v1/chat/completions", 
              json={
                  "model": "Phi-3 Mini Instruct",
                  "messages": [{"role": "user", "content": prompt}],
                  "max_tokens": 10,
                  "temperature": 0.28
                  })
   r = r.json()
   return r['choices'][0]['message']['content']


def main():
    conn = sqlite3.connect("../data/forum.db")    

    c = conn.cursor()

    c.execute("""
              CREATE TABLE IF NOT EXISTS crimes (
               promp STRING,
               crime STRING
              )
            """
              )
    
    c.execute("SELECT title FROM posts;")

    while True:
        batch = c.fetchall()
        print(len(batch))
        if not batch:
            break
        prompts: list[tuple[str]] = create_prompts(batch)
        
        for prompt in prompts:
            crime = gpt4all_request(prompt[1])
            print(f"---\nPROMPT:\n{prompt[0]}\nCRIME:{crime}\n---\n")
            c.execute("""INSERT INTO crimes values (?,?)""", (prompt[0], crime))
            conn.commit()
    conn.close()
            
main()


61916
---
PROMPT:
♦·♦·♦ Läs detta innan du postar! ♦·♦·♦
CRIME: No crime.
---

---
PROMPT:
24åring knivskuren i Lund 11 mars
CRIME: Assault.
---

---
PROMPT:
Gruppvåldtäkt på 13-åring....
CRIME: Gruppvåldtäkt (Group
---

---
PROMPT:
Kvinna rånad och dödad i Malmö
CRIME: Infanticide.
---

---
PROMPT:
Stenkastning i Rinkeby mot polisen.
CRIME: Arson.
---

---
PROMPT:
bilbomb i centrala London
CRIME: Bomb threat.
---

---
PROMPT:
Vem är dörrvakten?
CRIME: No crime.
---

---
PROMPT:
Skyddsgruppen
CRIME: No crime.
---

---
PROMPT:
Vem vill spränga Allsången i luften?
CRIME: Bomb threat.
---

---
PROMPT:
Narkotikaliga på väg att sprängas i Västerås.
CRIME: Narcoterrorism.
---

---
PROMPT:
Vad kan det här ha varit för folk?
CRIME: No crime.
---

---
PROMPT:
Mord i Malmö
CRIME: Mord (Murder)
---

---
PROMPT:
Landsortskriminalitet
CRIME: No crime.
---

---
PROMPT:
Skottlossning i Luleå i natt. Mellan vilka gäng?
CRIME: Bomb threat.
---

---
PROMPT:
Mord i hallstahammar
CRIME: Mord (Murder)
---


KeyboardInterrupt: 